**Задание 1**
Скачайте с сайта https://grouplens.org/datasets/movielens/ датасет любого размера. Определите какому фильму было выставлено больше всего оценок 5.0.

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('ratings.csv')

In [93]:
ratings.query('rating == {}'.format(5))['movieId'].value_counts().head(1)

318    170
Name: movieId, dtype: int64

**Задание 2**
По данным файла power.csv посчитайте суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года. Не учитывайте в расчетах отрицательные значения quantity.

In [7]:
power = pd.read_csv('power.csv')

In [8]:
baltics = power.query('country == ["Latvia", "Lithuania", "Estonia"] and category == [4, 12, 21] and year >= 2005 and year <= 2010 and quantity > 0')
baltics.quantity.sum()

240580.0

**Задание 3**
Выберите страницу любого сайта с табличными данными. Импортируйте таблицы в pandas dataframe.
Примеры страниц (необязательно брать именно эти):
https://fortrader.org/quotes
https://www.finanz.ru/valyuty/v-realnom-vremeni

In [14]:
page_url = 'https://www.finanz.ru/valyuty/v-realnom-vremeni'
course = pd.read_html(page_url, attrs = {'class': 'quote_list'}, encoding='utf-8')
course[:5]

[    Unnamed: 0              Рубль  Предложение  Спрос  Предыдущее закрытие  \
 0          NaN            GBP/RUB      1014880      -              1011254   
 1          NaN            EUR/RUB       916250      -               912570   
 2          NaN            USD/RUB       779796      -               776813   
 3          NaN            JPY/RUB        07407      -                07390   
 4          NaN            CNY/RUB       115962      -               115681   
 5          NaN            CHF/RUB       853877      -               850555   
 6          NaN               Евро  Предложение  Спрос  Предыдущее закрытие   
 7          NaN            EUR/USD        11750      -                11749   
 8          NaN            EUR/GBP        09028      -                09027   
 9          NaN            EUR/JPY      1236955      -              1235000   
 10         NaN            EUR/CHF        10730      -                10729   
 11         NaN         Доллар США  Предложение  Спр

**Задание 4**
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class


In [24]:
def classifies(mark):
        if mark <= 2:
            return 'низкий рейтинг'
        elif mark <= 4:
            return 'средний рейтинг'
        elif mark <= 4.5 or mark <= 5:
            return 'высокий рейтинг'

In [25]:
ratings['class'] = rating['rating'].apply(classifies)


In [27]:
ratings.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


**Задание 5**
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.
Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [77]:
keywords = pd.read_csv('keywords.csv',encoding='utf-8')
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [81]:
def region(reg):
    if reg in geo_data['Центр']:
        return 'Центр'
    elif reg in geo_data['Северо-Запад']:
        return 'Северо-Запад'
    elif reg in geo_data['Дальний Восток']:
        return 'Дальний Восток'
    else:
        return 'undefined'

In [96]:
keywords['region'] = keywords['keyword'].apply(region)
keywords.head()


,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [97]:
keywords[(keywords['region']=='Центр')]

,keyword,shows,region
1259,москва,153050,Центр
6075,ярославль,41720,Центр
11785,тула,23898,Центр


**Задание 6**
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [51]:
ratings = pd.read_csv('ratings.csv')

In [52]:
Ratings=ratings.merge(movies, on=["movieId"], how='inner')

In [53]:
def production_year(title):
    for year in years:
        if str(year) in title:
            return year
    return 1900

In [54]:
years = []
for year in range(1950,2010):
    years.append(year)

In [70]:
Ratings['year'] = Ratings['title'].apply(production_year)


In [89]:
Allratings = Ratings.pivot_table(index=['year'], values = 'rating', aggfunc='mean')
Allratings=Allratings.sort_values(by = ['rating'], ascending = False)
Allratings[:50]


,rating
year,
1957,4.014241
1972,4.011136
1952,4.000000
1954,3.994220
1951,3.983539
1974,3.978704
1962,3.952446
1950,3.915254
1977,3.905786


In [76]:
Allratings[51:]

,rating
year,
1999,3.471692
1992,3.466754
2001,3.460052
2002,3.456297
1990,3.449169
2005,3.448434
2003,3.444777
1996,3.426600
1997,3.415764


Прямой зависимости для выражения "с ростом года выпуска фильма его средний рейтинг становится ниже" нет. Если судить по данному выражению, то максимальный рейтинг должен быть в 1900 году(то есть раньше рассматриваемого периода), а минимальный в 2000-х годах. Рейтинг 1900 значительно ниже предполагаемого места. Утверждение не истинное.